In [35]:
# !pip add httpx python-docx

from pathlib import Path
from tempfile import gettempdir
from os import environ, getenv
import httpx
from bs4 import BeautifulSoup
from docx import Document
from docx.shared import Inches
import re
import pandas as pd

In [16]:
d = {
    'dim00692.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198055',
    'dim00693.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198063',
    'dim00694.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198064',
    'dim00571.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198052',
    'dim00572.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198054',
    'dim00573.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198053',
    'dim00695.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198065',
    'dim00696.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198068',
    'dim00697.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198071',
    'dim00698.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198072',
    'dim00699.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198083',
    'dim00700.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198084',
    'dim00701.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198085',
    'dim00702.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198087',
    'dim00703.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198086',
    'dim00704.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198088',
    'dim00705.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198089',
    'dim00706.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198090',
    'dim00707.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198091',
    'dim00708.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198092',
    'dim00709.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198093',
    'dim00710.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198095',
    'dim00711.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198094',
    'dim00712.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198096',
    'dim00713.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198097',
    'dim00714.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198098',
    'dim00715.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198101',
    'dim00716.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198102',
    'dim00717.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198103',
    'dim00718.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198100',
    'dim00719.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198099',
    'dim00720.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198104',
    'dim720-A.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198105',
    'dim00721.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198106',
    'dim00722.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198107',
    'dim00723.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198108',
    'dim00724.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198109',
    'dim00725.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198110',
    'dim00726.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198114',
    'dim00727.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198111',
    'dim00728.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198112',
    'dim00729.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198113',
    'dim00730.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198115',
    'dim00731.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198116',
    'dim731-A.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198118',
    'dim00732.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198119',
    'dim00733.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198120',
    'dim00734.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198121',
    'dim00735.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198123',
    'dim00736.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198122',
    'dim00737.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198124',
    'dim00738.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198125',
    'dim00739.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198126',
    'dim00740.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198127',
    'dim00741.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198128',
    'dim00742.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198129',
    'dim00743.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198130',
    'dim00744.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198131',
    'dim00745.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198132',
    'dim00746.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198133',
    'dim00747.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198134',
    'dim00748.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198135',
    'dim00749.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198136',
    'dim00574.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198056',
    'dim00575.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198060',
    'dim00576.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198058',
    'dim00577.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198062',
    'dim00578.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198059',
    'dim00579.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198057',
    'dim00580.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198061',
    'dim00583.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198067',
    'dim00584.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198066',
    'dim00587.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198069',
    'dim00588.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198070',
    'dim00590.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198073',
    'dim00591.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198077',
    'dim00592.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198078',
    'dim00593.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198074',
    'dim00594.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198079',
    'dim00595.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198076',
    'dim00596.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198075',
    'dim00597.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198080',
    'dim00598.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198081',
    'dim00600.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198082',
    'lim00581.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=207078',
    'lim00582.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=207080',
    'lim00585.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=207076',
    'lim00586.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=207083',
    'lim00589.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198146',
    'lim00599.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=207084',
    'lim00602.docx': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=207075',
}

In [3]:
def get_content(url: str) -> list:
    """Donwload content web."""
    req = httpx.get(url)
    content: str = req.text
    soup = BeautifulSoup(content, 'html5lib')
    result = soup.select_one(
        'div.corpo_texto_noticia'
    ).contents  # type: ignore[union-attr]
    return [re.sub('[\n\t]+', '', line.strip()) for line in result if len(line) > 1 and isinstance(line, str)]


In [13]:
# output = Path('/content/drive') / 'MyDrive' / 'projetos' / 'clbr'
output = Path.home().joinpath('projetos', 'cej_saj', 'acervo-legis-docx')
output

PosixPath('/home/user/projetos/cej_saj/acervo-legis-docx')

In [14]:
def dir_output(ano: str, parent: Path = output) -> Path:
  """Criar diretório de gravação dos arquivos."""
  directory = parent.joinpath(str(ano))
  directory.mkdir(parents=True, exist_ok=True)
  return directory

In [15]:
def docx_basic(fout: Path = None, content: list = None) -> bool:
    """Criar um docx básico."""
    fout = fout or Path(getenv('HANDLER_DOCX_FILENAME'))
    document = Document()
    document.add_heading(content.pop(0).replace('- ', ''))
    p = document.add_paragraph('')
    p.add_run(content.pop(0)).italic = True
    for conteudo in content:
        document.add_paragraph(conteudo)
    document.save(fout.as_posix())
    return fout.is_file()


In [22]:
def raspagem (ano: int, d: dict, start: int = 0, final: int = None):
    """"""
    for filename, link in list(d.items())[start:final]:
      print(filename, end=' .. ')
      docx_basic(dir_output(ano)/filename, get_content(link)[3:])
      print('ok')

In [23]:
raspagem(1850, d)

dim00692.docx .. ok
dim00693.docx .. ok
dim00694.docx .. ok
dim00571.docx .. ok
dim00572.docx .. ok
dim00573.docx .. ok
dim00695.docx .. ok
dim00696.docx .. ok
dim00697.docx .. ok
dim00698.docx .. ok
dim00699.docx .. ok
dim00700.docx .. ok
dim00701.docx .. ok
dim00702.docx .. ok
dim00703.docx .. ok
dim00704.docx .. ok
dim00705.docx .. ok
dim00706.docx .. ok
dim00707.docx .. ok
dim00708.docx .. ok
dim00709.docx .. ok
dim00710.docx .. ok
dim00711.docx .. ok
dim00712.docx .. ok
dim00713.docx .. ok
dim00714.docx .. ok
dim00715.docx .. ok
dim00716.docx .. ok
dim00717.docx .. ok
dim00718.docx .. ok
dim00719.docx .. ok
dim00720.docx .. ok
dim720-A.docx .. ok
dim00721.docx .. ok
dim00722.docx .. ok
dim00723.docx .. ok
dim00724.docx .. ok
dim00725.docx .. ok
dim00726.docx .. ok
dim00727.docx .. ok
dim00728.docx .. ok
dim00729.docx .. ok
dim00730.docx .. ok
dim00731.docx .. ok
dim731-A.docx .. ok
dim00732.docx .. ok
dim00733.docx .. ok
dim00734.docx .. ok
dim00735.docx .. ok
dim00736.docx .. ok


In [58]:
filein = Path.home().joinpath('projetos', 'jedi', 'academia-jedi', 'data_files', 'csv').rglob('CLBR*')
# list(filein)

In [59]:
for file in filein:
    print(file)

/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1849.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1829.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1838.csv
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1842.csv
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1848.csv
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1834.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1841.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1847.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1836.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1840.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1851.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1839.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1832.CSV
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1833.csv
/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1845

In [60]:
df = pd.read_csv('/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1829.CSV', delimiter=';')
df

,ano,filename,url
0,1829,dim18290728-001.docx,https://www.diariodasleis.com.br/legislacao/fe...
1,1829,dim18290729-002.docx,https://www.diariodasleis.com.br/legislacao/fe...
2,1829,dim18290803-003.docx,https://www.diariodasleis.com.br/legislacao/fe...
3,1829,dim18290829-004.docx,https://www.diariodasleis.com.br/legislacao/fe...
4,1829,dim18290831-005.docx,https://www.diariodasleis.com.br/legislacao/fe...
...,...,...,...
74,1829,dim18291125-076.docx,https://www.diariodasleis.com.br/legislacao/fe...
75,1829,dim18291125-077.docx,https://www.diariodasleis.com.br/legislacao/fe...
76,1829,dim18291127-078.docx,https://www.diariodasleis.com.br/legislacao/fe...
77,1829,dim18291222-079.docx,https://www.diariodasleis.com.br/legislacao/fe...


In [61]:
df.to_dict(orient='records')

[{'ano': 1829,
  'filename': 'dim18290728-001.docx',
  'url': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203962'},
 {'ano': 1829,
  'filename': 'dim18290729-002.docx',
  'url': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203961'},
 {'ano': 1829,
  'filename': 'dim18290803-003.docx',
  'url': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203958'},
 {'ano': 1829,
  'filename': 'dim18290829-004.docx',
  'url': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203955'},
 {'ano': 1829,
  'filename': 'dim18290831-005.docx',
  'url': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203953'},
 {'ano': 1829,
  'filename': 'dim18290831-006.docx',
  'url': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203954'},
 {'ano': 1829,
  'filename': 'dim18290905-007.docx',
  'url': 'https://www.diariodasleis.com.br/legislacao/federal/exibe_art

In [63]:
for idx, (ano, filename, url) in df.iterrows():
    print(ano, filename, url)

1829 dim18290728-001.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203962
1829 dim18290729-002.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203961
1829 dim18290803-003.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203958
1829 dim18290829-004.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203955
1829 dim18290831-005.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203953
1829 dim18290831-006.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203954
1829 dim18290905-007.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203951
1829 dim18290905-008.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203950
1829 dim18290905-009.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203952
1829 dim18290918-010.docx https://www.diarioda

In [64]:
docx_basic(dir_output(1829)/'dim18291229-080.docx', get_content('https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203914')[3:])

True

In [65]:
for idx, (ano, filename, url) in df.iterrows():
    print(ano, filename, url, end=' .. ')
    docx_basic(dir_output(ano)/filename, get_content(url)[3:])
    print('ok')

1829 dim18290728-001.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203962 .. ok
1829 dim18290729-002.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203961 .. ok
1829 dim18290803-003.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203958 .. ok
1829 dim18290829-004.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203955 .. ok
1829 dim18290831-005.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203953 .. ok
1829 dim18290831-006.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203954 .. ok
1829 dim18290905-007.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203951 .. ok
1829 dim18290905-008.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=203950 .. ok
1829 dim18290905-009.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=20395

In [73]:
display(df)
pd.read_csv('/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1843.CSV', delimiter=';')

,ano,filename,url
0,1829,dim18290728-001.docx,https://www.diariodasleis.com.br/legislacao/fe...
1,1829,dim18290729-002.docx,https://www.diariodasleis.com.br/legislacao/fe...
2,1829,dim18290803-003.docx,https://www.diariodasleis.com.br/legislacao/fe...
3,1829,dim18290829-004.docx,https://www.diariodasleis.com.br/legislacao/fe...
4,1829,dim18290831-005.docx,https://www.diariodasleis.com.br/legislacao/fe...
...,...,...,...
74,1829,dim18291125-076.docx,https://www.diariodasleis.com.br/legislacao/fe...
75,1829,dim18291125-077.docx,https://www.diariodasleis.com.br/legislacao/fe...
76,1829,dim18291127-078.docx,https://www.diariodasleis.com.br/legislacao/fe...
77,1829,dim18291222-079.docx,https://www.diariodasleis.com.br/legislacao/fe...


,ano,filename,url
0,1843,dim00261.docx,https://www.diariodasleis.com.br/legislacao/fe...
1,1843,dim00262.docx,https://www.diariodasleis.com.br/legislacao/fe...
2,1843,dim00263.docx,https://www.diariodasleis.com.br/legislacao/fe...
3,1843,dim00264.docx,https://www.diariodasleis.com.br/legislacao/fe...
4,1843,dim00266.docx,https://www.diariodasleis.com.br/legislacao/fe...
...,...,...,...
67,1843,dim00335.docx,https://www.diariodasleis.com.br/legislacao/fe...
68,1843,dim00336.docx,https://www.diariodasleis.com.br/legislacao/fe...
69,1843,lim00281.docx,https://www.diariodasleis.com.br/legislacao/fe...
70,1843,lim00282.docx,https://www.diariodasleis.com.br/legislacao/fe...


In [75]:
df.append(pd.read_csv('/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1843.CSV', delimiter=';')).reset_index()

/tmp/ipykernel_3368/1258784838.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(pd.read_csv('/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1843.CSV', delimiter=';')).reset_index()


,index,ano,filename,url
0,0,1829,dim18290728-001.docx,https://www.diariodasleis.com.br/legislacao/fe...
1,1,1829,dim18290729-002.docx,https://www.diariodasleis.com.br/legislacao/fe...
2,2,1829,dim18290803-003.docx,https://www.diariodasleis.com.br/legislacao/fe...
3,3,1829,dim18290829-004.docx,https://www.diariodasleis.com.br/legislacao/fe...
4,4,1829,dim18290831-005.docx,https://www.diariodasleis.com.br/legislacao/fe...
...,...,...,...,...
146,67,1843,dim00335.docx,https://www.diariodasleis.com.br/legislacao/fe...
147,68,1843,dim00336.docx,https://www.diariodasleis.com.br/legislacao/fe...
148,69,1843,lim00281.docx,https://www.diariodasleis.com.br/legislacao/fe...
149,70,1843,lim00282.docx,https://www.diariodasleis.com.br/legislacao/fe...


In [88]:
filein = Path.home().joinpath('projetos', 'jedi', 'academia-jedi', 'data_files', 'csv').rglob('CLBR*')
for file in filein:
    df = pd.read_csv(file, delimiter=';')
    print(file)
    for idx, (ano, filename, url) in df.iterrows():
        print(ano, filename, url, end=' .. ')
        docx_basic(dir_output(ano)/filename, get_content(url)[3:])
        print('ok')


/home/user/projetos/jedi/academia-jedi/data_files/csv/CLBR1849.CSV
1849 dim00572.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198169 .. ok
1849 dim00573.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198170 .. ok
1849 dim00574.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198171 .. ok
1849 dim00575.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198172 .. ok
1849 dim576-A.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198174 .. ok
1849 dim00576.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198173 .. ok
1849 dim00577.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198177 .. ok
1849 dim00578.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=198176 .. ok
1849 dim00579.docx https://www.diariodasleis.com.br/legislacao/federal/exibe_artigo.php?ifl=1

In [85]:
df

,ano,filename,url
0,1849,dim00572.docx,https://www.diariodasleis.com.br/legislacao/fe...
1,1849,dim00573.docx,https://www.diariodasleis.com.br/legislacao/fe...
2,1849,dim00574.docx,https://www.diariodasleis.com.br/legislacao/fe...
3,1849,dim00575.docx,https://www.diariodasleis.com.br/legislacao/fe...
4,1849,dim576-A.docx,https://www.diariodasleis.com.br/legislacao/fe...
...,...,...,...
86,1849,dim00659.docx,https://www.diariodasleis.com.br/legislacao/fe...
87,1849,dim00660.docx,https://www.diariodasleis.com.br/legislacao/fe...
88,1849,dim00661.docx,https://www.diariodasleis.com.br/legislacao/fe...
89,1849,dim00662.docx,https://www.diariodasleis.com.br/legislacao/fe...
